# 2 数据预处理
## 2.1 生成数据
创建一个人工数据集，并存储在C:/Users/xinyu/Desktop/myjupyter/data/class.csv中：

In [28]:
import os

os.makedirs(os.path.join('C:/Users/xinyu/Desktop/myjupyter', 'data'), exist_ok=True)#exist_ok=True确保存在路径C:/Users/xinyu/Desktop/myjupyter
data_file = os.path.join('C:/Users/xinyu/Desktop/myjupyter', 'data', 'class.csv')  #csv文件的意思是每一行是一个文件，每一个域用逗号隔开
with open(data_file, 'w') as f:  #将数据按行写进文件
    f.write('group, name, score, whether_pass\n')  #列名
    f.write('1, ZhangSan, 78, yes\n')  # 每行表示一个数据样本
    f.write('3, LiSi, 55, no\n')
    f.write('NA, WangWu,66, yes\n')
    f.write('5, ZhaoLiu,NA,NA\n')  #写入NA的时候要注意字符不能空格，否则写入的就是NA这个字符而不是NANI类型的missing value
    
#从创建的xls文件中加载原始数据集：导入pandas包并调用read_csv函数（一般读取csv文件用padans库）
import pandas as pd

data = pd.read_csv(data_file)
print(data)
data  #也可以不print，就会更好看，用html形式展现出来

   group       name   score  whether_pass
0    1.0   ZhangSan    78.0           yes
1    3.0       LiSi    55.0            no
2    NaN     WangWu    66.0           yes
3    5.0    ZhaoLiu     NaN           NaN


,group,name,score,whether_pass
0,1.0,ZhangSan,78.0,yes
1,3.0,LiSi,55.0,no
2,NaN,WangWu,66.0,yes
3,5.0,ZhaoLiu,NaN,NaN


## 2.2 处理缺失值
“NaN”项代表缺失值，处理缺失值典型的方法包括插值和删除。
下面的column1_3为数值缺失的，column4为内容缺失的。
首先对数值缺失的，取均值：

In [32]:
column1_3, column4 = data.iloc[:, ::2], data.iloc[:, -1]  #取出第一列、第三列，取出第四列。iloc就是index location
column1_3 = column1_3.fillna(column1_3.mean())  #对第一列、第三列NaN的数据取平均值
column1_3  #就是对数值域进行处理了

,group,score
0,1.0,78.000000
1,3.0,55.000000
2,3.0,66.000000
3,5.0,66.333333


其次对内容缺失的，将“NaN”视为一个类别：

In [33]:
column4 = pd.get_dummies(column4, dummy_na=True)
column4  #从结果可以看出，四组的分别是yes，no，yes，NaN0.

,no,yes,NaN
0,0,1,0
1,1,0,0
2,0,1,0
3,0,0,1


## 2.3 转换为张量格式
现在column1_3和column4中的所有条目都是数值类型，它们可以转换为张量格式。

In [41]:
import torch
x, y = torch.tensor(column1_3.values), torch.tensor(column4.values)
x, y

(tensor([[ 1.0000, 78.0000],
         [ 3.0000, 55.0000],
         [ 3.0000, 66.0000],
         [ 5.0000, 66.3333]], dtype=torch.float64),
 tensor([[0, 1, 0],
         [1, 0, 0],
         [0, 1, 0],
         [0, 0, 1]], dtype=torch.uint8))

上面是float64，传统的python一般默认浮点数为float64，但计算很慢，深度学习通常用32位浮点数